In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize



In [2]:
import requests

link = "https://www.4icu.org/us/"
f = requests.get(link)
# print(f.text)

# print(f.text)

import re

university_list = re.findall("<td><a href=\S+>(\D+)</a>", f.text)
for x in range(len(university_list)):
    curr = university_list[x]
    if curr.lower() == "columbia university in the city of new york":
        university_list[x] = curr[:19] 
    if "&amp" in curr:
        university_list[x].replace("&amp;", " ")
        # and_index = curr.index("&")
        # university_list[x] = curr[0:and_index] + " " + curr[and_index+5:]
    if "The" in curr:
        university_list[x] = curr[3:]
    if "," in curr:
        university_list[x].replace(",", "")

print(len(university_list))

top_400 = university_list[0:400]

# print(top_200)


1766


In [5]:
import requests

# link = "https://www.collegevine.com/schools/hub/overall/sheet"

link = "https://www.collegevine.com/schools/hub/data/static"

# avoid printing f.text; Large file
f = requests.get(link)


# "majorsCipCodes":["11.0701","14.0501","14.1001","14.1301","14.1901","16.0102","16.0104","16.0201",
# "16.0301","16.0302","16.0303","16.04","16.0402","16.0407","16.0408","16.0501","16.0502","16.0506",
# "16.0599","16.0701","16.09","16.0901","16.0902","16.0904","16.0905","16.1101","16.1102","16.12",
# "16.1203","16.1299","16.1301","16.1402","16.1501","23.0101","24.0101","24.0103","26.0101","26.0202",
# "26.0404","26.0406","26.0599","26.1303","26.1503","27.0101","27.0301","27.0501","3.0103","30.1301",
# "30.1701","30.1801","38.0101","38.0201","39.0601","40.0202","40.0203","40.0299","40.0501","40.0508",
# "40.0599","40.0601","40.0801","42.0101","42.2701","42.2704","45.0101","45.0201","45.0301","45.0601",
# "45.0603","45.1001","45.1101","5.0104","5.0105","5.0108","5.011","5.0112","5.0201","5.0207","5.0208",
# "5.0209","5.0299","50.0101","50.0501","50.0601","50.0702","50.0703","50.0901","51.221","54.0101",
# "54.0103","54.0104","54.0199"],"name":"Harvard University","numFreshmen"

# "majorsCipCodes":["11.0103","11.0201","11.0701","11.0801","11.1099","11.9999","13.1001","13.1203",
# "13.121","13.1302","13.1303","13.1305","13.1306","13.1307","13.1308","13.1311","13.1312","13.1314",
# "13.1316","13.1318","13.1322","13.1325","13.133","14.0701","14.0801","14.1001","14.1801","14.1901",
# "14.3501","15.0201","15.0303","15.0702","15.0805","15.1201","16.0102","16.0901","16.0902","16.0905",
# "19.0707","23.0101","23.1302","23.1303","23.1304","24.0101","24.0102","26.0101","26.0202","27.0101",
# "3.0104","30.1101","31.0501","31.0505","38.0101","38.0201","40.0201","40.0501","40.0601","40.0699",
# "40.0801","42.0101","43.0104","43.0106","44","44.0701","45.0101","45.0201","45.0601","45.0701","45.0799",
# "45.0901","45.1001","45.1101","5.0101","5.0102","5.0201","50.0399","50.0501","50.0702","50.0703","50.0901",
# "50.0902","50.0903","50.0904","50.1001","51","51.0001","51.0602","51.0701","51.0702","51.0899","51.0908",
# "51.1005","51.1504","51.2201","51.3101","51.3801","51.9999","52.0201","52.0205","52.0301","52.0304","52.0601",
# "52.0801","52.0901","52.1001","52.1002","52.1101","52.1201","52.1206","52.1302","52.1399","52.1401","52.1403",
# "52.1902","54.0101","9.0401","9.0701","9.0903","9.0906"],"name":"Youngstown State University","numFreshmen"

import re

# university_list = re.findall("\"name\":\"([^\"]+)\",\"numFreshmen\"", f.text)
university_list = re.findall("\"rankings\":.+?\"bucket\":\"(\d*)\",\"key\":\"overall\".+?\"slug\":\"(.+?)\"", f.text)

# \"slug\":\"(.+?)\"
# print(university_list[0:10])

# "\"rankings\":.+?\"order\":(\d*)(?=.+?\"key\"=\"overall\")\"slug\":\"(.+?)\""


# # \"majorsCipCodes\".+\"
# # \"numFreshmen\"

# # ([^,\"])

# # for x in range(len(university_list)):
# #     curr = university_list[x]

print(university_list[0])



('305', 'abilene-christian-university')


In [24]:
gap = 0

for x in range(len(university_list)):
    university_list[x] = [int(university_list[x][0]), university_list[x][1]]
    if x+1 != int(university_list[x][0]) - gap:
        gap += 1
        print(university_list[x-1:x+1])

print(len(sorted(university_list, key = lambda pair: pair[0])))

university_list = sorted(university_list, key = lambda pair: pair[0])

top_400 = university_list[0:400]

# print(len(university_list))

# top_400 = university_list[0:400]

# print(top_400)

[[287, 'bryant-university'], [289, 'milwaukee-school-of-engineering-msoe']]
[[403, 'ohio-wesleyan-university'], [405, 'brigham-young-university-hawaii']]
[[647, 'university-of-indianapolis'], [649, 'franklin-college']]
[[676, 'st-edward-s-university'], [678, 'high-point-university']]
[[799, 'university-of-nebraska-omaha-uno'], [801, 'university-of-toledo']]
[[852, 'rocky-mountain-college-rmc'], [854, 'hardin-simmons-university']]
[[867, 'johnson-c-smith-university'], [869, 'spring-arbor-university']]
[[1053, 'tabor-college'], [1055, 'wells-college']]
[[1064, 'hope-international-university'], [1066, 'university-of-northwestern-st-paul']]
[[1191, 'concordia-college-new-york'], [1193, 'methodist-university']]
[[1264, 'notre-dame-de-namur-university'], [1266, 'felician-university']]
[[1349, 'dakota-state-university'], [1351, 'southern-university-at-new-orleans']]
1506


In [6]:
testtext = "\"majorsCipCodes\":[\"11.0701\", \"11.0701\"],\"name\":\"Harvard University\",\"numFreshmen\""

testlist = re.findall("\"majorsCipCodes\".+\"name\":\"(.*)\",\"numFreshmen\"", testtext)

print(testlist)

['Harvard University']


In [25]:
# get links for all schools

college_links = []

for school in top_400:
    name = school[1]
    college_links.append("https://www.collegevine.com/schools/" 
                            + name.replace(" ", "-") 
                            +  "/essay-prompts")

print(college_links[0])

https://www.collegevine.com/schools/stanford-university/essay-prompts


In [26]:
# access links

potential_issues = []

for link in college_links:
    temp_f = requests.get(link)
    prompts = re.findall("<p>(.+)</p>", temp_f.text)

    if prompts == []:
        potential_issues.append(link)


# print(college_links[333])

# temp_f = requests.get(college_links[323])

# prompts = re.findall("<p>(.+)</p>", temp_f.text)
# print(prompts)


KeyboardInterrupt: 

In [ ]:
len(potential_issues)

print(potential_issues)

In [12]:
for x in top_400:
    if " in " in x[1]:
        print(x)

In [ ]:
for link in college_links:
    curr_file = requests.get(link)
    prompts = re.findall("<p>(.+)</p>", curr_file.text)

    if prompts == []:
        potential_issues.append(link)



In [ ]:
# do this
# conda env export > environment.yml